In [46]:
"""Processes the FETAL .dcm dataset into .npy files."""

import glob, os, sys, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from lxml import etree

from pprint import pprint
import gdcm
import pylab

import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir

In [47]:
"""The FETAL dataset comes into the following format:
FetalLung/
    FetalLungNormal/ 
        LC20141215330/
            series/
                IM-0182-0001-d.dcm
                ...
        LC20141217317/
            AX_SSFSE_BODY/
                IM-0991-0001-d.dcm
                ...
            COR_SSFSE_BODY/
                IM-0991-0001-d.dcm
                ...
            SAG_SSFSE_BODY/
                IM-0992-0001-d.dcm
                ...
        ...
    FetalLungAbnormal/ 
        patient/
            LC20150128274/
                AX_SSFSExr_BODY/
                    IM-0782-0001-d.dcm
                    ...
                Cor_SSFSExr_BODY/
                    IM-0781-0001-d.dcm
                    ...
                SAG_SSFSExr_BODY/
                    IM-0783-0001-d.dcm
                    ...
            ...

The output .npy files are placed in the following directory:
FETAL/
    processed/
        0-0002.npy
        ...
with each file as [class]-[scan_id].npy where class is 
0 (abnormal) or 1 (normal).
"""

DEBUG = False

# Directory holding the decompressed normal fetal images
normal_directory = '/../home/mazin/FetalLung/FetalLungNormal'

# Directory holding the decompressed abnormal fetal images
abnormal_directory = '/../home/mazin/FetalLung/FetalLungAbnormal/patient'

# Directory output for the the processed .npy files
output_directory = './data/FETAL/processed'

assert os.path.isdir(normal_directory), "Couldn't find the dataset at {}".format(input_directory)
assert os.path.isdir(abnormal_directory), "Couldn't find the dataset at {}".format(input_directory)

In [48]:
def getPathsForSubjects(class_path):
    subject_paths = glob.glob("%s/*" % class_path)
    subject_ids = [path.split('/')[-1] for path in subject_paths]
    return subject_ids, subject_paths

def getPathsForScans(subject_path):
    scan_paths = glob.glob("%s/*" % subject_path)
    scan_types = [path.split('/')[-1] for path in scan_paths]
    return scan_types, scan_paths

In [49]:
def getUniqueSeriesPaths(scan_path):
    """
    Group together .dcm by their SeriesInstanceUID attribute

    Returns a dict of unique SeriesInstanceUID => [.dcm paths]
    """
    seriesDict = {}
    dicom_paths = glob.glob("%s/*.dcm" % scan_path)
    for dicom_path in dicom_paths:
        dicom_obj = pydicom.read_file(dicom_path)
        if dicom_obj.SeriesInstanceUID in seriesDict:
            seriesDict[dicom_obj.SeriesInstanceUID].append(dicom_path)
        else:
            seriesDict[dicom_obj.SeriesInstanceUID] = [dicom_path]
    return seriesDict

In [50]:
def getSeriesOrientation(series_paths):
    num_dicoms = len(series_paths)
    
    file = series_paths[0]
    r = gdcm.ImageReader()
    r.SetFileName(file)
    if not r.Read():
      sys.exit(1)
    ir = r.GetImage()

    # orientation
    dircos =  ir.GetDirectionCosines()
    t = gdcm.Orientation.GetType(tuple(dircos))
    orientation = gdcm.Orientation.GetLabel(t)
    
    return orientation
    
def load_dicoms(dicom_paths, displacement):
    num_dicoms = len(dicom_paths)

    dcm_dict = get_dcm_dict(dicom_paths[0])
    ref_ds = dcm_dict['ds']

    # Create the structure for the 3-D volume
    data = np.zeros([num_dicoms, ref_ds.Rows, ref_ds.Columns])

    for dicom_path in dicom_paths:
        dcm_dict = get_dcm_dict(dicom_path)
        ds = dcm_dict['ds']
        if ds.Rows != ref_ds.Rows or ds.Columns != ref_ds.Columns:
            print("Invalid scan size: " +  str([ds.Rows, ds.Columns]))
            continue    
        slice_num = int(ds.InstanceNumber) - displacement
        if slice_num >= len(data):
            print("slice_num: ", slice_num )
            slice_num = len(data) - 1
        if dcm_dict['raw'].shape != data[slice_num, :, :].shape:
            print("Incorrect shape: " + str([dcm_dict['raw'].shape, data[slice_num, :, :].shape]))
            continue
        data[slice_num,:,:] = dcm_dict['raw']
        
    data = np.asarray(data, dtype='int16')

    # sliceBitmap = data[10]
    # plt.imshow(sliceBitmap, cmap=cm.Greys_r)
    # plt.show()

    return data

In [51]:
def convertDicomsToMatrix(class_label, series_paths, series_idx):
    """
    Converts and saves the .dcm files to .npy numpy matrix files in the 
    class directory.
    """
    print("Converting dicom %s to numpy" % series_idx)
    output_file_path = getOutputFilePath(class_label, series_idx) # without orientation
    
    # Get the orientation of the .dcm files in the series (COR/AXL/SAG)
    orientation = getSeriesOrientation(series_paths)
    print(orientation)
    
    # Convert to 3D numpy matrix and save to output directory
    numpy_matrix = getNumpyMatrix(series_paths)

    # If the file exists, ask the user whether they want to overwrite.
    # Exit out of the function without saving if they do not want to overwrite.
#     if (len(sys.argv) <= 1 or sys.argv[1] != '--overwrite') and os.path.exists(output_file_path):
#         try:
#             query_question = "File %s already exists. Would you like to overwrite it?" % output_file_path
#             if not query_yes_no(query_question):
#                 return
#         except EOFError as e:
#             pass
    
    print("Matrix Dimensions: %s" % str(numpy_matrix.shape))
    # np.save(output_file_path, numpy_matrix)

    print("Saved %s" % output_file_path)
    print()

def getNumpyMatrix(series_paths):
    """
    Collects .dcm files into a 3D numpy matrix.
    """
    # Get the dicom paths for this scan_id
    displacement, max_frame_num = seriesPathToMinMaxFrameNumber(series_paths)
    if DEBUG:
        print(displacement)
        print(max_frame_num)
        print()

    # raw_matrix is the 3d array of the dicom slices in order
    print(scan_path)
    raw_matrix = load_dicoms(series_paths, displacement)
    return raw_matrix

# def getDicomPaths(scan_path, scan_id):
#     """
#     Looks in the scan directory for all .dcm files.
#     """
#     dicom_paths = glob.glob("%s/IM-%s*.dcm" % (scan_path, scan_id))
#     if len(dicom_paths) == 0:
#         print(scan_path, scan_id)
#     return dicom_paths

def seriesPathToMinMaxFrameNumber(dicom_paths):
    allFrameNumbers = []
    for dicom_path in dicom_paths:
        frameNumber = getFrameNumberForDicomPath(dicom_path)
        allFrameNumbers.append(frameNumber)
    return min(allFrameNumbers), max(allFrameNumbers)

def getFrameNumberForDicomPath(dicom_path):
    dicom_obj = pydicom.read_file(dicom_path)
    if DEBUG:
        print("FrameOfReferenceUID: ", dicom_obj.FrameOfReferenceUID) 
        print("InstanceNumber: ", dicom_obj.InstanceNumber) 
        print("SliceLocation: ", dicom_obj.SliceLocation) 
        print("WindowCenter: ", dicom_obj.WindowCenter) 
        print("WindowWidth: ", dicom_obj.WindowWidth) #ImageOrientationPatient
        print("SeriesUID: ", dicom_obj.SeriesInstanceUID)
        print("ImagePositionPatient: ", dicom_obj.ImagePositionPatient)
    frameNumber = dicom_obj.InstanceNumber
    return frameNumber

def load_dicoms(dicom_paths, displacement):
    num_dicoms = len(dicom_paths)

    dcm_dict = get_dcm_dict(dicom_paths[0])
    ref_ds = dcm_dict['ds']

    # Create the structure for the 3-D volume
    data = np.zeros([num_dicoms, ref_ds.Rows, ref_ds.Columns])

    for dicom_path in dicom_paths:
        dcm_dict = get_dcm_dict(dicom_path)
        ds = dcm_dict['ds']
        if ds.Rows != ref_ds.Rows or ds.Columns != ref_ds.Columns:
            print("Invalid scan size: " +  str([ds.Rows, ds.Columns]))
            continue    
        slice_num = int(ds.InstanceNumber) - displacement
        if slice_num >= len(data):
            print("slice_num: ", slice_num )
            slice_num = len(data) - 1
        if dcm_dict['raw'].shape != data[slice_num, :, :].shape:
            print("Incorrect shape: " + str([dcm_dict['raw'].shape, data[slice_num, :, :].shape]))
            continue
        data[slice_num,:,:] = dcm_dict['raw']
        
    data = np.asarray(data, dtype='int16')

    # sliceBitmap = data[10]
    # plt.imshow(sliceBitmap, cmap=cm.Greys_r)
    # plt.show()

    return data

def get_dcm_dict(dicom_path):
    """
    lung_raw: the raw output from the dicom file
    lung_im: the grayscale array
    ds: the original ds format if you want to work directly with it
    """
    ds = pydicom.read_file(dicom_path)
    lung = ds.pixel_array
    lung_raw = np.asarray(lung, dtype='float')
    lung_im = (lung_raw/lung_raw.max()) * 255.0
    return {'raw': lung_raw, 'grayscale': lung_im, 'ds': ds}

def getOutputFilePath(class_label, series_idx):
    """
    Returns the .npy file name with [label]-[series_idx].npy
    (e.g. 0-1127.npy)
    """
    outputFileName = "%s_%s.npy" % (class_label, str(series_idx).zfill(4))
    outputFilePath = os.path.join(output_directory, outputFileName)
    return outputFilePath

In [52]:
#### Utility Functions ####
def query_yes_no(question, default="yes"):
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

In [ ]:
class_paths = [normal_directory, abnormal_directory]
series_idx = 0
for class_idx, class_path in enumerate(class_paths): # abnormal, normal
    subject_ids, subject_paths = getPathsForSubjects(class_path) 
    for subject_id, subject_path in zip(subject_ids, subject_paths): 
        scan_types, scan_paths = getPathsForScans(subject_path) # e.g. series/, COR/, AXL/, etc.
        for scan_type, scan_path in zip(scan_types, scan_paths):
            series_dict = getUniqueSeriesPaths(scan_path) # get the unique scans in the class
            for series_id, series_paths in series_dict.items():
                convertDicomsToMatrix(class_idx, series_paths, series_idx)
                series_idx += 1


Converting dicom %s to numpy 0
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/SAG SSFSE BODY
Matrix Dimensions: (24, 256, 256)
Saved ./data/FETAL/processed/0_0000.npy

Converting dicom %s to numpy 1
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/COR SSFSE BODY
Matrix Dimensions: (15, 256, 256)
Saved ./data/FETAL/processed/0_0001.npy

Converting dicom %s to numpy 2
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/AX SSFSE BODY
Matrix Dimensions: (27, 256, 256)
Saved ./data/FETAL/processed/0_0002.npy

Converting dicom %s to numpy 3
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/SAG SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ./data/FETAL/processed/0_0003.npy

Converting dicom %s to numpy 4
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/COR SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ./data/FETAL/processed/0_0004.npy

Converting dicom %s to numpy 5
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/AX SSFSE BODY LARGE
Matrix

Converting dicom %s to numpy 46
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (36, 512, 512)
Saved ./data/FETAL/processed/0_0046.npy

Converting dicom %s to numpy 47
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (50, 512, 512)
Saved ./data/FETAL/processed/0_0047.npy

Converting dicom %s to numpy 48
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (38, 256, 256)
Saved ./data/FETAL/processed/0_0048.npy

Converting dicom %s to numpy 49
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/Cor SSFSEx BODY
Matrix Dimensions: (17, 512, 512)
Saved ./data/FETAL/processed/0_0049.npy

Converting dicom %s to numpy 50
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/Sag SSFSEx BODY
Matrix Dimensions: (15, 512, 512)
Saved ./data/FETAL/processed/0_0050.npy

Converting dicom %s to numpy 51
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/AX SSFSEx BODY

/../home/mazin/FetalLung/FetalLungNormal/LC20150225201/COR SSFSExr BODY
Matrix Dimensions: (24, 512, 512)
Saved ./data/FETAL/processed/0_0092.npy

Converting dicom %s to numpy 93
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150225201/AX SSFSExr BODY
Matrix Dimensions: (39, 512, 512)
Saved ./data/FETAL/processed/0_0093.npy

Converting dicom %s to numpy 94
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/SAG SSFSEx BODY
Matrix Dimensions: (19, 512, 512)
Saved ./data/FETAL/processed/0_0094.npy

Converting dicom %s to numpy 95
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/COR SSFSEx BODY
Matrix Dimensions: (19, 512, 512)
Saved ./data/FETAL/processed/0_0095.npy

Converting dicom %s to numpy 96
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/AX SSFSEx BODY
Matrix Dimensions: (32, 512, 512)
Saved ./data/FETAL/processed/0_0096.npy

Converting dicom %s to numpy 97
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC1077409/SAG SSFSE BODY
Matrix Dimensions: (16, 51

Converting dicom %s to numpy 138
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/COR SSFSEx BH
Matrix Dimensions: (21, 512, 512)
Saved ./data/FETAL/processed/0_0138.npy

Converting dicom %s to numpy 139
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/Ax SSFSEx
Matrix Dimensions: (32, 512, 512)
Saved ./data/FETAL/processed/0_0139.npy

Converting dicom %s to numpy 140
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/AX SSFSEx BH
Matrix Dimensions: (31, 512, 512)
Saved ./data/FETAL/processed/0_0140.npy

Converting dicom %s to numpy 141
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC961597/SAG SSFSE BODY
Matrix Dimensions: (21, 512, 512)
Saved ./data/FETAL/processed/0_0141.npy

Converting dicom %s to numpy 142
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC961597/COR SSFSE BODY
Matrix Dimensions: (22, 512, 512)
Saved ./data/FETAL/processed/0_0142.npy

Converting dicom %s to numpy 143
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC961597/AX SSFSE BODY
Matrix 

Matrix Dimensions: (21, 512, 512)
Saved ./data/FETAL/processed/0_0183.npy

Converting dicom %s to numpy 184
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2089655/COR SSFSE BODY
Matrix Dimensions: (20, 512, 512)
Saved ./data/FETAL/processed/0_0184.npy

Converting dicom %s to numpy 185
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2089655/AX SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ./data/FETAL/processed/0_0185.npy

Converting dicom %s to numpy 186
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170607382/series
Matrix Dimensions: (22, 512, 512)
Saved ./data/FETAL/processed/0_0186.npy

Converting dicom %s to numpy 187
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170607382/series
Matrix Dimensions: (34, 512, 512)
Saved ./data/FETAL/processed/0_0187.npy

Converting dicom %s to numpy 188
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170607382/series
Matrix Dimensions: (15, 512, 512)
Saved ./data/FETAL/processed/0_0188.npy

Converting dicom %s to numpy 189
AXIAL

...

Matrix Dimensions: (42, 512, 512)
Saved ./data/FETAL/processed/0_0955.npy

Converting dicom %s to numpy 956
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2493592/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ./data/FETAL/processed/0_0956.npy

Converting dicom %s to numpy 957
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2493592/COR SSFSE BODY
Matrix Dimensions: (18, 512, 512)
Saved ./data/FETAL/processed/0_0957.npy

Converting dicom %s to numpy 958
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2493592/AX SSFSE BODY
Matrix Dimensions: (25, 512, 512)
Saved ./data/FETAL/processed/0_0958.npy

Converting dicom %s to numpy 959
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20141224224/Cor SSFSEx
Matrix Dimensions: (34, 512, 512)
Saved ./data/FETAL/processed/0_0959.npy

Converting dicom %s to numpy 960
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20141224224/SAG SSFSEx
Matrix Dimensions: (28, 512, 512)
Saved ./data/FETAL/processed/0_0960.npy

Converting dicom %s to num

Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1001.npy

Converting dicom %s to numpy 1002
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170724201/series
Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1002.npy

Converting dicom %s to numpy 1003
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150318209/Cor SSFSEx BODY
Matrix Dimensions: (29, 512, 512)
Saved ./data/FETAL/processed/0_1003.npy

Converting dicom %s to numpy 1004
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150318209/SAG SSFSEx BODY
Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1004.npy

Converting dicom %s to numpy 1005
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150318209/AX SSFSEx BODY
Matrix Dimensions: (57, 512, 512)
Saved ./data/FETAL/processed/0_1005.npy

Converting dicom %s to numpy 1006
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170712341/series
Matrix Dimensions: (54, 512, 512)
Saved ./data/FETAL/processed/0_1006.npy

Converting dic

Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1047.npy

Converting dicom %s to numpy 1048
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170913387/series
Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1048.npy

Converting dicom %s to numpy 1049
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170913387/series
Matrix Dimensions: (48, 512, 512)
Saved ./data/FETAL/processed/0_1049.npy

Converting dicom %s to numpy 1050
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170913387/series
Matrix Dimensions: (61, 512, 512)
Saved ./data/FETAL/processed/0_1050.npy

Converting dicom %s to numpy 1051
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160824368/series
Matrix Dimensions: (30, 512, 512)
Saved ./data/FETAL/processed/0_1051.npy

Converting dicom %s to numpy 1052
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160824368/series
Matrix Dimensions: (29, 256, 256)
Saved ./data/FETAL/processed/0_1052.npy

Converting dicom %s to numpy 1053
AXIAL


/../home/mazin/FetalLung/FetalLungNormal/LC20170315428/series
Matrix Dimensions: (56, 512, 512)
Saved ./data/FETAL/processed/0_1093.npy

Converting dicom %s to numpy 1094
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170315428/series
Matrix Dimensions: (33, 512, 512)
Saved ./data/FETAL/processed/0_1094.npy

Converting dicom %s to numpy 1095
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160815266/series
Matrix Dimensions: (26, 512, 512)
Saved ./data/FETAL/processed/0_1095.npy

Converting dicom %s to numpy 1096
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160815266/series
Matrix Dimensions: (36, 512, 512)
Saved ./data/FETAL/processed/0_1096.npy

Converting dicom %s to numpy 1097
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160815266/series
Matrix Dimensions: (20, 512, 512)
Saved ./data/FETAL/processed/0_1097.npy

Converting dicom %s to numpy 1098
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC699838/SAG SSFSE BODY
Matrix Dimensions: (23, 512, 512)
Saved ./data/FET

Matrix Dimensions: (18, 512, 512)
Saved ./data/FETAL/processed/0_1138.npy

Converting dicom %s to numpy 1139
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC810914/SAG SSFSE BODY
Matrix Dimensions: (22, 512, 512)
Saved ./data/FETAL/processed/0_1139.npy

Converting dicom %s to numpy 1140
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC810914/COR SSFSE BODY
Matrix Dimensions: (23, 512, 512)
Saved ./data/FETAL/processed/0_1140.npy

Converting dicom %s to numpy 1141
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC810914/AX SSFSE BODY LARGE
Matrix Dimensions: (45, 512, 512)
Saved ./data/FETAL/processed/0_1141.npy

Converting dicom %s to numpy 1142
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC1537365/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ./data/FETAL/processed/0_1142.npy

Converting dicom %s to numpy 1143
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC1537365/COR SSFSE BODY
Matrix Dimensions: (14, 512, 512)
Saved ./data/FETAL/processed/0_1143.npy

Converting dicom %